In [1]:
! pip install finance-datareader

In [2]:
import FinanceDataReader as fdr
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import itertools
import random

from sklearn.preprocessing import MinMaxScaler, StandardScaler

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense,  GRU, Concatenate, Dropout, LSTM, SimpleRNN, TimeDistributed, Bidirectional
from tensorflow.keras import Sequential, Input, Model, layers, optimizers
from tensorflow.keras.optimizers import SGD, Adam

In [3]:
random.seed(1234)
np.random.seed(1234)
os.environ["PYTHONHASHSEED"] = str(1234)
tf.random.set_seed(1234)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
path = '/content/drive/MyDrive/주식 종가 예측/open/'
list_name = 'stock_list.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


# Public

In [24]:
start_date = '20210104'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

In [12]:
def RNN_model(X_train, y_train, X_val, y_val):

  model = Sequential()
  model.add(SimpleRNN(units = 1, input_shape = (X_train.shape[1], 1), activation = 'tanh', return_sequences = True))
  model.add(SimpleRNN(units = 8, activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.125))
  model.add(SimpleRNN(units = 8, activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.125))
  model.add(SimpleRNN(units = 8, activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.125))
  model.add(SimpleRNN(units = 8, activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.125))
  model.add(SimpleRNN(units = 8, activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.125))
  model.add(SimpleRNN(units = 8, activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.125))

  model.add(TimeDistributed(Dense(units = 1)))

  model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'mean_absolute_error', metrics=['mae']) 

  es = EarlyStopping(monitor ='val_loss', mode ='min', verbose = 1, patience = 10)
  model_save_folder_path = '/content/drive/MyDrive/주식 종가 예측/RNN3/'
  model_path = model_save_folder_path + f'{code}.h5'
  mc = ModelCheckpoint(model_path, monitor ='val_loss', mode ='min', verbose = 1, save_best_only = True)

  model.fit(X_train, y_train, epochs = 100, batch_size = 2, verbose = 1, shuffle = False, callbacks = [es, mc],
                      validation_data=(X_val, y_val))

  return model

In [13]:
submission_name = 'sample_submission.csv'
submission = pd.read_csv(os.path.join(path,submission_name))

In [14]:
nmae_df = pd.DataFrame(columns = ['code', 'nmae'])

In [15]:
import warnings
warnings.filterwarnings("ignore")

for code in tqdm(stock_list['종목코드'].values):
  tf.random.set_seed(1234)
  
  # 해당 종목의 데이터 가져온 후 주 단위 데이터로 나타내기
  stock_price = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  stock_price = pd.merge(business_days, stock_price, how = 'outer')
  stock_price['weekday'] = stock_price.Date.apply(lambda x : x.weekday())
  stock_price['weeknum'] = stock_price.Date.apply(lambda x : x.strftime('%V'))
  stock_price.Close = stock_price.Close.ffill()
  stock_price = pd.pivot_table(data = stock_price, values = 'Close', columns = 'weekday', index = 'weeknum')

  X_train = stock_price.iloc[0:-12].to_numpy()
  X_val = stock_price.iloc[-12:-2].to_numpy()
  X_test = stock_price.iloc[-2].to_numpy()

  y_train = stock_price.iloc[1:-11].to_numpy()
  y_val = stock_price.iloc[-11:-1].to_numpy()
  y_test = stock_price.iloc[-1].to_numpy()

  # 스케일링 시행
  scaler = MinMaxScaler()

  X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(32, 5)
  X_val_scaled = scaler.fit_transform(X_val.reshape(-1, 1)).reshape(10, 5)
  X_test_scaled = scaler.fit_transform(X_test.reshape(-1, 1)).reshape(1, 5)

  y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(32, 5)
  y_val_scaled = scaler.fit_transform(y_val.reshape(-1, 1)).reshape(10, 5)
  y_test_scaled = scaler.fit_transform(y_test.reshape(-1, 1)).reshape(1, 5)

  X_train_scaled = np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1]))
  X_val_scaled = np.reshape(X_val_scaled, (X_val_scaled.shape[0], X_val_scaled.shape[1]))
  X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1]))
  
  # 모델 학습
  model = RNN_model(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled)
  model = keras.models.load_model(f"/content/drive/MyDrive/주식 종가 예측/RNN3/{code}.h5")

  # 예측
  y_pred = model.predict(X_test_scaled)
  y_pred = y_pred.reshape(1,5)
  y_pred = scaler.inverse_transform(y_pred)

  nmae = np.mean(np.abs(y_test - y_pred) / y_test)

  nmae_df = nmae_df.append({'code' : code, 'nmae' : nmae}, ignore_index = True)
  
  print(y_pred)
  print(nmae)
    
  for i in range (0, 5):
    submission.loc[i, code] = y_pred[0][i]

Output hidden; open in https://colab.research.google.com to view.

In [16]:
# nmae 평균
nmae_result = np.array(nmae_df['nmae'])

In [17]:
np.mean(nmae_result)

0.01729472269182923

# Private

In [18]:
start_date = '20210104'
end_date = '20211126'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

In [19]:
for code in tqdm(stock_list['종목코드'].values):
  
  # 해당 종목의 데이터 가져온 후 주 단위 데이터로 나타내기
  stock_price = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  stock_price = pd.merge(business_days, stock_price, how = 'outer')
  stock_price['weekday'] = stock_price.Date.apply(lambda x : x.weekday())
  stock_price['weeknum'] = stock_price.Date.apply(lambda x : x.strftime('%V'))
  stock_price.Close = stock_price.Close.ffill()
  stock_price = pd.pivot_table(data = stock_price, values = 'Close', columns = 'weekday', index = 'weeknum')

  X_test = stock_price.iloc[46].to_numpy()

  # 스케일링 시행
  scaler = MinMaxScaler()

  X_test_scaled = scaler.fit_transform(X_test.reshape(-1, 1)).reshape(1, 5)
  X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1]))
  
  # 모델 학습
  model = keras.models.load_model(f"/content/drive/MyDrive/주식 종가 예측/RNN3/{code}.h5")

  # 예측
  y_pred = model.predict(X_test_scaled)
  y_pred = y_pred.reshape(1,5)
  y_pred = scaler.inverse_transform(y_pred)

  print(y_pred)
    
  for i in range (0, 5):
    submission.loc[i + 5, code] = y_pred[0][i] # 임시로 채워주기 위한 용도. 최종 예측 시에는 바꿔주어야 함!

  0%|          | 1/370 [00:03<18:32,  3.02s/it]

[[74066.81  74212.29  74222.57  74172.62  74012.945]]


  1%|          | 2/370 [00:05<18:00,  2.94s/it]

[[117885.28  118416.055 118179.734 118151.42  118114.96 ]]


  1%|          | 3/370 [00:08<17:28,  2.86s/it]

...

[[6444.4204 6439.068  6430.8916 6392.249  6323.882 ]]


100%|█████████▉| 369/370 [17:06<00:02,  2.65s/it]

[[46510.504 48066.08  47901.566 47753.082 47761.832]]


100%|██████████| 370/370 [17:08<00:00,  2.78s/it]

[[47254.63  47088.348 47063.31  46841.965 46328.574]]


In [20]:
submission.isna().sum().sum()

0

In [21]:
submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29059.160156,34790.500000,60873.636719,147252.171875,112020.187500,17391.080078,49644.367188,86515.156250,106558.312500,692433.2500,51392.953125,232107.671875,32542.291016,58080.128906,16699.769531,2320.055176,26225.335938,5208.018555,322496.96875,50504.589844,13558.614258,32297.576172,9216.420898,7993.026855,30456.205078,147196.296875,47523.917969,88329.796875,43006.902344,236789.484375,283324.53125,80407.406250,100956.929688,33577.265625,29239.703125,33068.421875,143043.812500,211627.734375,98880.859375,...,39624.296875,107015.468750,275612.50000,12894.326172,166801.453125,175428.328125,7157.744141,7423.260254,34742.941406,13434.116211,87488.226562,49841.449219,56018.937500,25266.394531,267447.625000,595432.25000,682895.1250,22656.294922,51805.867188,72414.734375,29362.292969,119278.960938,36959.152344,13400.349609,3326.352295,39262.738281,18855.679688,64513.660156,96424.968750,5674.284668,49107.726562,52730.335938,83422.406250,35307.167969,25358.746094,51051.367188,359962.75000,251085.437500,26431.703125,17880.105469
1,2021-11-02,29288.392578,34908.246094,60963.085938,147767.109375,114767.796875,17592.314453,49899.980469,86397.492188,106556.437500,692939.3125,51471.269531,232324.031250,32597.345703,57977.234375,16818.251953,2323.060059,26327.498047,5215.316406,322125.56250,50641.585938,13625.822266,32334.152344,9270.610352,7997.408691,30416.234375,147137.546875,47505.273438,88966.781250,43199.625000,237241.718750,283499.62500,80498.265625,101180.882812,33609.320312,29506.677734,32955.898438,143307.781250,211253.453125,99083.085938,...,39442.871094,106934.765625,275486.56250,12887.968750,166760.093750,175325.687500,7118.230469,7440.614746,34724.296875,13465.408203,90106.437500,49805.921875,55883.304688,25288.990234,267406.375000,598919.06250,684003.9375,22511.187500,51932.132812,72479.187500,29597.437500,119498.906250,36846.191406,13420.619141,3338.433838,39197.171875,18874.228516,65429.804688,97550.093750,5673.493164,48866.792969,53184.617188,83584.828125,35285.332031,25589.550781,51024.406250,357180.18750,253031.218750,26702.648438,17867.044922
2,2021-11-03,29443.205078,34599.343750,60832.730469,147675.671875,121685.984375,17578.470703,49849.566406,86474.265625,106643.093750,693199.6875,51485.671875,232610.593750,32585.312500,58166.125000,16875.943359,2320.184326,26312.363281,5207.567383,322812.09375,50261.343750,13627.534180,32438.412109,9308.611328,8003.528320,30390.140625,147261.156250,47516.894531,86825.093750,43119.136719,236153.218750,283360.43750,80428.437500,101150.898438,33541.093750,29555.048828,33141.363281,143118.859375,213966.500000,99114.335938,...,39321.257812,106984.804688,276579.06250,12887.102539,166727.609375,173929.796875,7243.875977,7447.066406,34741.636719,13489.337891,93566.398438,49846.097656,55857.554688,25468.916016,267521.031250,589029.62500,670978.0000,22579.246094,51989.382812,74623.578125,31507.503906,119187.195312,38034.835938,13512.613281,3344.895020,38996.351562,18844.380859,66157.468750,97811.484375,5671.152344,48885.945312,53545.425781,83274.945312,35617.246094,25603.154297,51674.496094,370657.53125,257455.406250,26779.953125,17888.453125
3,2021-11-04,29353.425781,34126.179688,60686.550781,147386.234375,117208.375000,17152.343750,49830.691406,86720.328125,106707.593750,689527.0625,51597.718750,232248.171875,32551.925781,58162.300781,16866.128906,2310.468750,25771.117188,5196.947266,321919.25000,49718.546875,13604.916992,3246

In [22]:
submission.to_csv("/content/drive/MyDrive/주식 종가 예측/submission_simpleRNN_ver3.csv", index = False)